<a href="https://colab.research.google.com/github/projecte-aina/rag_notebook/blob/main/RAGDemo_LlamaIndex_Flor6.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace LLM - Flor6.3b

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙. (Based on an [original tutorial](https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/customization/llms/SimpleIndexDemo-Huggingface_camel.ipynb) by the LlamaIndex Team )

In [ ]:
%pip install huggingface_hub llama-index-embeddings-huggingface llama-index-llms-huggingface

In [ ]:
!pip install llama-index langchain

In [ ]:
import logging
import sys
from llama_index.core import Settings

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-small",
    embed_batch_size=2
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#### Download Data

In [ ]:
!mkdir -p 'data/estrategia/'
!wget 'https://politiquesdigitals.gencat.cat/web/.content/00-arbre/economia/catalonia-ai/Estrategia_IA_Catalunya_VFinal_CAT.pdf' -O 'data/estrategia/Estrategia_IA_Catalunya.pdf'

--2024-03-14 13:05:34--  https://politiquesdigitals.gencat.cat/web/.content/00-arbre/economia/catalonia-ai/Estrategia_IA_Catalunya_VFinal_CAT.pdf
Resolving politiquesdigitals.gencat.cat (politiquesdigitals.gencat.cat)... 23.9.93.11, 2600:1406:5800:380::988, 2600:1406:5800:3aa::988
Connecting to politiquesdigitals.gencat.cat (politiquesdigitals.gencat.cat)|23.9.93.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘data/estrategia/Estrategia_IA_Catalunya.pdf’

data/estrategia/Est     [ <=>                ]   4.59M  26.9MB/s    in 0.2s    

2024-03-14 13:05:35 (26.9 MB/s) - ‘data/estrategia/Estrategia_IA_Catalunya.pdf’ saved [4809573]



#### Load documents, build the VectorStoreIndex

In [ ]:
# load documents, creates one document per page from the PDF
documents = SimpleDirectoryReader("./data/estrategia/").load_data()

#index documents segmented into chunks of max. 512 tokens
Settings.chunk_size = 512
Settings.chunk_overlap = 100
index = VectorStoreIndex.from_documents(documents)

In [ ]:
#check one of the indexed documents/pages (page 1)
documents[1]

Document(id_='eea40f99-6723-4dfb-9514-098b1d103edc', embedding=None, metadata={'page_label': '2', 'file_name': 'Estrategia_IA_Catalunya.pdf', 'file_path': '/content/data/estrategia/Estrategia_IA_Catalunya.pdf', 'file_type': 'application/pdf', 'file_size': 4809573, 'creation_date': '2024-03-14', 'last_modified_date': '2023-06-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=' \n \n \nL’Estratègia d’Intel·ligència Artificial de Catalunya  \n1  \n \n \n \n L’ESTRAT ÈGIA D ’INTEL ·LIGÈNCIA ARTIFICAL DE  CATALUNYA  \n \nAutor : \nGeneralitat de Catalunya  \nDepartament de Pol ítiques Digitals i Administraci ó Pública   \nSecretaria de Polítiques Digitals    \n \nCol·laboradors:  \nGrup de treball per a la definici ó de l’Estrat ègia d ’intel ·ligència art

#### Set up LLM model and prompt

In [ ]:
from llama_index.core import PromptTemplate

# dolly format prompt template
query_prompt = PromptTemplate(
    "### Instruction:\n{query_str}\n\n### Context\n{context_str}### Answer:\n"
)

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.25, "do_sample": True},
    tokenizer_name="projecte-aina/FLOR-6.3B-Instructed",
    model_name="projecte-aina/FLOR-6.3B-Instructed",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### Query Index

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()

#set prompt template to use Dolly format
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": query_prompt}
)

response = query_engine.query("Qué és l’Estratègia d’intel·ligència artificial de Catalunya?")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


L'Estratègia d'Intel·ligència Artificial de Catalunya és un document que estableix les línies mestres de la política de la Generalitat de Catalunya en matèria d'intel·ligència artificial.#


In [ ]:
print(response)

L'Estratègia d'Intel·ligència Artificial de Catalunya és un document que estableix les línies mestres de la política de la Generalitat de Catalunya en matèria d'intel·ligència artificial.#


#### Query Index - Streaming

In [ ]:
query_engine = index.as_query_engine(streaming=True)
query_engine.update_prompts({"response_synthesizer:text_qa_template": query_prompt})

In [ ]:
response_stream = query_engine.query("En Europa, quin país és capdavanter en intel·ligència artificial?")
response_stream.print_response_stream()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


El Regne Unit.#